In [1]:
import numpy as np
import torch
import torch.nn as nn
import csv
import os
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import time

In [2]:
from setup_dataset import *
from setup_model import *
from setup_model_types import *

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
data = chunk_dataset(match_list=[1],
                     start_frame = 1,
                     end_frame = 1030
                    )

Successfully loaded NPZ.
Do you want to use GPU (y/n)n
Selected CPU


C:\Users\alece\Desktop\UnB\tcc\rnn_enduro\2-pytorch_train\setup_dataset.py:133: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  X_train = np.array(X_train)
C:\Users\alece\Desktop\UnB\tcc\rnn_enduro\2-pytorch_train\setup_dataset.py:134: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  Y_train = np.array(Y_train)


In [5]:
data.padSequence()

In [6]:
original_shape = data.data.shape
original_shape

torch.Size([124, 30, 100, 100])

In [7]:
data.data = data.data.reshape(-1, 1, 100, 100)

In [8]:
data.data.shape, data.targets.shape, data.seq_len.shape

(torch.Size([3720, 1, 100, 100]), torch.Size([124, 30, 9]), torch.Size([124]))

In [9]:
device = data.device
input_size = data.data.shape[2]
output_size = len(data.available_targets)
match_list = data.match_list
start_frame = data.start_frame
end_frame = data.end_frame

In [10]:
data.seq_len.shape

torch.Size([124])

In [11]:
class CNNLSTMModel(nn.Module):
    def __init__(self, device, input_size, output_size, hidden_dim, n_layers):
        super(CNNLSTMModel, self).__init__()
        
        self.device = device
        self.input_size = input_size
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.init_hidden()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=3, kernel_size=(3,3))
        self.conv2 = nn.Conv2d(3, 3, (10,10))
        self.activation = nn.ReLU()
        self.bnorm = nn.BatchNorm2d(num_features=3)
        self.pool = nn.MaxPool2d(kernel_size=(2,2))

        # output = (input - filter + 1) / stride
        # convolução 1: (100 - 3 + 1) / 1 = 98x98
        # pooling 1: 49x49
        # convolução 2: (45 - 10 + 1) / 1 = 36x36
        # pooling 2: 18x18
        # 18 * 18 * 3
        self.lstm = nn.LSTM(49*49*3, hidden_dim, n_layers, batch_first=True)  
        self.fc = nn.Linear(hidden_dim, output_size)
        self.out = nn.Softmax()
    
    def forward(self, x):
        
        hidden = self.init_hidden()

        x = self.pool(self.bnorm(self.activation(self.conv1(x))))
        # x = self.pool(self.bnorm(self.activation(self.conv2(x))))

        x = x.view(original_shape[0], original_shape[1], -1)
        
        print(x.shape, data.seq_len.shape)
        
        x = pack_padded_sequence(x, data.seq_len, batch_first=True, enforce_sorted=False)
        
        print(x)
        
        pad_embed_pack_lstm = self.lstm(x, hidden)
        pad_embed_pack_lstm_pad = pad_packed_sequence(pad_embed_pack_lstm[0], batch_first=True)
        
        outs, _ = pad_embed_pack_lstm_pad
        
        out = outs.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        out = self.out(out)
        
        return out
        
    def init_hidden(self):
        # the weights are of the form (nb_layers, batch_size, nb_lstm_units)
        hidden_a = torch.randn(1, self.input_size, self.hidden_dim)
        hidden_b = torch.randn(1, self.input_size, self.hidden_dim)

        if self.device.type == 'cuda':
            hidden_a = hidden_a.cuda()
            hidden_b = hidden_b.cuda()

        hidden_a = Variable(hidden_a)
        hidden_b = Variable(hidden_b)

        return (hidden_a, hidden_b)

In [12]:
model_structure = ModelStructure(device, input_size, output_size, match_list, start_frame, end_frame)

Number of hidden neurons: 200
Number of epochs: 5000
Choose type of RNN model:
1 - Simple RNN
2 - LSTM
3 - CNN
type: 3
write a observations without space and punctuations:teste
models/CNN_teste_m1_f1to1030_epoch5000_H200
ATTENTION! folder not created. Training informations will overwrite the existing one


In [13]:
model = CNNLSTMModel(device=model_structure.device, 
                     input_size=model_structure.data_size, 
                     output_size=model_structure.output_size, 
                     hidden_dim=model_structure.hidden_neurons, 
                     n_layers=1)

In [14]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

In [15]:
mymodel = model
X_train = data.data 
Y_train = data.targets

In [16]:
def get_acc(predicted, target):
    
    predicted = torch.argmax(predicted, axis=1)
    target = torch.argmax(target, axis=1)

    acc = np.mean(predicted == target)
    
    return float(acc)

In [17]:
train_loss_arr = np.array([])
train_acc_arr = np.array([])

In [18]:
X_train.shape, Y_train.shape

(torch.Size([3720, 1, 100, 100]), torch.Size([124, 30, 9]))

In [19]:
start_time_processing = time.time()

loss_file = open(model_structure.path + '/' + "loss_file.txt", "w")
first_time = True

best_loss = 1
first_epoch = True

for epoch in range(1, model_structure.n_epochs + 1):

    mymodel.train()

    optimizer.zero_grad()
    output = mymodel(X_train)
    loss = criterion(output, Y_train.view(-1,data.target_size[2]).float())
    loss.backward()
    optimizer.step()
        
    if epoch%10 == 0:

        train_loss_arr = np.append(train_loss_arr, loss.item())
        train_acc_arr  = np.append(train_acc_arr, model_structure.get_acc(output, Y_train.reshape(-1, len(data.available_targets))))
        
        loss_file.write("Epoch: {}/{}-------------------------------------------\n".format(epoch, model_structure.n_epochs))
        loss_file.write("Train -> Loss: {:.15f} Acc: {:.15f}\n".format(train_loss_arr[-1], train_acc_arr[-1]))
            
        print("Epoch: {}/{}-------------------------------------------".format(epoch, model_structure.n_epochs))
        print("Train -> Loss: {:.15f} Acc: {:.15f}".format(train_loss_arr[-1], train_acc_arr[-1]))
        
        if train_loss_arr[-1] < best_loss:
            state = { 'epoch': epoch + 1, 'state_dict': mymodel.state_dict(),
                      'optimizer': optimizer.state_dict(), 'losslogger': loss.item(), }
            torch.save(state, model_structure.path + '/' + model_structure.name)
            best_loss = loss.item()
        else:
            print("model not saved")
            
loss_file.write("--- %s seconds ---" % (time.time() - start_time_processing))
loss_file.close()
np.savez(model_structure.path + '/' + "train_loss_arr", train_loss_arr)
print("--- %s seconds ---" % (time.time() - start_time_processing))

torch.Size([124, 30, 7203]) torch.Size([124])
PackedSequence(data=tensor([[-0.0753, -0.0753, -0.0753,  ..., 15.1196, -0.0823, -0.0823],
        [-0.0753, -0.0753, -0.0753,  ..., -0.0823, -0.0823, -0.0823],
        [-0.0753, -0.0753, -0.0753,  ..., -0.0823, -0.0823, -0.0823],
        ...,
        [-0.0753, -0.0753, -0.0753,  ..., -0.0823, -0.0823, -0.0823],
        [-0.0753, -0.0753, -0.0753,  ..., -0.0823, -0.0823, -0.0823],
        [-0.0753, -0.0753, -0.0753,  ..., -0.0823, -0.0823, -0.0823]],
       grad_fn=<PackPaddedSequenceBackward>), batch_sizes=tensor([124, 119, 111,  92,  77,  71,  60,  52,  46,  40,  35,  29,  26,  24,
         19,  18,  15,  14,  11,   6,   4,   4,   4,   4,   3,   3,   2,   2,
          2,   1]), sorted_indices=tensor([ 13,   0,   5,  43,   7,  19, 106,  12, 114,  32, 116,   4,  47,  45,
         74,   1,  97,  90,  88,   9,  56,  34,  80,  95,  49,  76,  58,  41,
         53,  21,  14,   6, 100,  96, 113,  75,  72,  17,   2,  55,  83,  71,
         86,  15,

RuntimeError: INDICES element is out of DATA bounds, id=106 axis_dim=100

In [ ]:
data.seq_len

In [ ]:
data.data.shape

In [ ]:
data.targets.shape

In [ ]:
output.shape

In [ ]:
Y_train.view(-1,data.target_size[2]).shape

In [ ]:
data.seq_len.shape

In [ ]:
data.padSequence()

In [ ]:
data.packPaddedSequence()

In [ ]:
data.data[0].shape